In [1]:
import freqml as fm
from freqml import *
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
df = fm.read("ETH")

In [6]:
df.shape

(400201, 7)

In [3]:
df_TB = df.bars.TB()

### discover promising points

In [7]:
%%time
promising_points = fm.CUSUM(df)

CPU times: user 6.95 s, sys: 3.66 ms, total: 6.96 s
Wall time: 6.95 s


In [21]:
promising_points.shape

(27891, 1)

In [8]:
promising_points.sample(3)

,datetime
14872,2020-04-16 07:35:27.445000+00:00
26723,2020-04-16 16:04:34.231000+00:00
17300,2020-04-16 08:27:55.929000+00:00


### label dataset using triple barrier 

In [26]:
%%time
df_labels = fm.labeling(df_TB, promising_points[:20000], up=2, down=2, lapse=3)

CPU times: user 4min 6s, sys: 200 ms, total: 4min 7s
Wall time: 4min 6s


In [32]:
df_labels.sample(5)

,start,end,size,side,duration,touch_time
2262,155.68,156.06,0.38,0,00:03:11.753000,2020-04-15 12:43:25.002000+00:00
10340,160.75,162.71,1.96,1,00:02:38.006000,2020-04-16 07:18:39.358000+00:00
11194,162.87,164.60,1.73,1,00:00:35.149000,2020-04-16 07:19:13.434000+00:00
2137,156.71,156.25,0.46,0,00:03:07.085000,2020-04-15 12:12:41.222000+00:00
8460,153.28,153.11,0.17,0,00:04:04.720000,2020-04-16 05:23:43.778000+00:00


In [28]:
df_labels.shape

(20000, 6)

### meta-label dataset using triple barrier

In [33]:
predictions = pd.DataFrame(np.random.choice([-1, 1], df_labels.shape[0]))

In [35]:
%%time
df_meta_labels = fm.meta_labeling(df_TB, promising_points[:20000], predictions)

CPU times: user 4min 8s, sys: 208 ms, total: 4min 8s
Wall time: 4min 8s


In [37]:
df_meta_labels.sample(5)

,start,end,size,pred_side,true_side,take,duration,touch_time
8628,154.87,154.85,0.02,1,0,0,00:05:07.289000,2020-04-16 06:28:03.731000+00:00
12134,167.8,166.19,1.61,1,-1,0,00:00:16.170000,2020-04-16 07:20:00.035000+00:00
15742,164.35,165.44,1.09,1,1,1,00:00:40.405000,2020-04-16 08:00:23.758000+00:00
5153,152.64,152.71,0.07,1,0,0,00:05:04.239000,2020-04-15 23:18:52.378000+00:00
10667,160.97,162.26,1.29,-1,1,0,00:01:31.941000,2020-04-16 07:18:22.209000+00:00


In [38]:
df_meta_labels.shape

(20000, 8)